In [1]:
%matplotlib inline

In [2]:
from __future__ import division
from numpy.random import randn, uniform, normal
import random
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import calendar
from datetime import date, timedelta


In [3]:
# 2010 is the last available year right now
years = range(1880, 2017)

pieces = []
columns = ['name', 'sex', 'births']

for year in years:
    path = 'names/yob%d.txt' % year
    frame = pd.read_csv(path, names=columns)

    frame['year'] = year
    pieces.append(frame)

# Concatenate everything into a single DataFrame
names = pd.concat(pieces, ignore_index=True)

In [7]:
pieces = []
for year, group in names.groupby(['year', 'sex']):
    pieces.append(group.sort_values(by='births', ascending=False)[:1000])
top1000 = pd.concat(pieces, ignore_index=True)

In [8]:
top1000.head()

,name,sex,births,year
0,Mary,F,7065,1880
1,Anna,F,2604,1880
2,Emma,F,2003,1880
3,Elizabeth,F,1939,1880
4,Minnie,F,1746,1880


In [9]:
def firstName(year,sex):
    df = top1000[top1000.year == year]
    df = df[df.sex == sex]
    total = float(np.sum(df.births))
   
    df = df.sort_values(by=['births'], ascending=False)
    df["freq"] = df.births.apply(lambda x: float(x)/total)
    df["bound"] = df.freq.cumsum()
    return df
firstName(1968,"M").tail()

,name,sex,births,year,freq,bound
177872,Octavio,M,55,1968,0.000033,0.999869
177873,Torrance,M,55,1968,0.000033,0.999902
177874,Raynard,M,55,1968,0.000033,0.999935
177875,Raleigh,M,55,1968,0.000033,0.999967
177876,Prince,M,55,1968,0.000033,1.000000


In [10]:
bar = firstName(1973,"F")
bar.head()

,name,sex,births,year,freq,bound
185877,Jennifer,F,62454,1973,0.048249,0.048249
185878,Amy,F,26965,1973,0.020832,0.069081
185879,Michelle,F,26930,1973,0.020805,0.089886
185880,Kimberly,F,23532,1973,0.018180,0.108066
185881,Lisa,F,22667,1973,0.017512,0.125578


In [11]:
bar.bound.searchsorted(0.07)[0], bar.bound.searchsorted(0.11)[0], bar.bound.searchsorted(0.01)[0]

(2, 4, 0)

In [12]:
def rowName(df,prob):
    i = df.bound.searchsorted(prob)[0]
    row = df.iloc[i]
    return row["name"]

rowName(bar,0.11)

'Lisa'

In [13]:
def makeFirstList(year,sex,count):
    bar = firstName(year,sex)
    ret = []
    for x in range(count):
        prob = uniform()
        name = rowName(bar,prob)
        ret.append(name)
        
    return ret

makeFirstList(1912,"M",10)

['Donald',
 'Tommie',
 'Jeff',
 'Joseph',
 'Ralph',
 'Horace',
 'Francis',
 'Tracy',
 'Sidney',
 'Joe']

### Last Names
from the [census 1990](https://www.census.gov/topics/population/genealogy/data/1990_census/1990_census_namefiles.html)

In [14]:
path = 'surnames/1990.txt'
lastName = pd.read_csv(path,  delim_whitespace=True , names = ["uName","freq","c","Rank"])
lastName.head()

,uName,freq,c,Rank
0,SMITH,1.006,1.006,1
1,JOHNSON,0.810,1.816,2
2,WILLIAMS,0.699,2.515,3
3,JONES,0.621,3.136,4
4,BROWN,0.621,3.757,5


In [15]:
lastName["name"] = lastName.uName.apply(lambda x: str(x).title())
total = float(np.sum(lastName.freq))


lastName["freqReal"] = lastName.freq.apply(lambda x: float(x)/total)
lastName["bound"] = lastName.freqReal.cumsum()
lastName.head()

,uName,freq,c,Rank,name,freqReal,bound
0,SMITH,1.006,1.006,1,Smith,0.012640,0.012640
1,JOHNSON,0.810,1.816,2,Johnson,0.010177,0.022817
2,WILLIAMS,0.699,2.515,3,Williams,0.008783,0.031599
3,JONES,0.621,3.136,4,Jones,0.007802,0.039402
4,BROWN,0.621,3.757,5,Brown,0.007802,0.047204


In [16]:
rowName(lastName,0.02)

'Johnson'

In [17]:
def randomFullName(year,gender,count):
    aval = makeFirstList(year,gender,count)
    midList = makeFirstList(year,gender,count)
    ret = []
    for i in range(len(aval)):
        first = aval[i]
        middle = midList[i]
        prob = uniform()
        last = rowName(lastName,prob)
        name = first + " " + middle[0] + ". " + last
        ret.append( {"first":first,"middle":middle,"gender":gender,"last":last, "name":name})
    
        
    return ret
randomFullName(1986,"M",10)

[{'first': 'Vincent',
  'gender': 'M',
  'last': 'Cambell',
  'middle': 'Adam',
  'name': 'Vincent A. Cambell'},
 {'first': 'Joshua',
  'gender': 'M',
  'last': 'Robbs',
  'middle': 'Joshua',
  'name': 'Joshua J. Robbs'},
 {'first': 'Stephon',
  'gender': 'M',
  'last': 'Lee',
  'middle': 'James',
  'name': 'Stephon J. Lee'},
 {'first': 'Courtney',
  'gender': 'M',
  'last': 'Bradley',
  'middle': 'Douglas',
  'name': 'Courtney D. Bradley'},
 {'first': 'Kory',
  'gender': 'M',
  'last': 'Denning',
  'middle': 'Jay',
  'name': 'Kory J. Denning'},
 {'first': 'Bruce',
  'gender': 'M',
  'last': 'Williams',
  'middle': 'Jacob',
  'name': 'Bruce J. Williams'},
 {'first': 'Daniel',
  'gender': 'M',
  'last': 'Farr',
  'middle': 'Edward',
  'name': 'Daniel E. Farr'},
 {'first': 'Julian',
  'gender': 'M',
  'last': 'Moriarty',
  'middle': 'Charles',
  'name': 'Julian C. Moriarty'},
 {'first': 'Gregory',
  'gender': 'M',
  'last': 'Day',
  'middle': 'Michael',
  'name': 'Gregory M. Day'},
 {'fi

## year

In [27]:
def randomDate(year):
    firstJan = date(year,1,1)
    d = firstJan + timedelta(days = random.randint(0, 365 if calendar.isleap(firstJan.year) else 364))
    return {"year":year,"month":d.month,"day":d.day}
    
randomDate(2015)

{'day': 3, 'month': 8, 'year': 2015}

In [46]:
def getYears(yearMean,std,count,gender):
    (years,yearcounts) = np.unique(normal(yearMean,std,count).astype(np.int32),return_counts=True)
    
    ret = []
    
   
   
    for i in range(len(years)):
        year = int(years[i])
        ycount =  yearcounts[i]
        names = randomFullName(year,gender,ycount)


        for j in range(len(names)):
            pep = names[j]
            pep["birth"] = randomDate(year)
            ret.append(pep)
    
    arr = np.arange(len(ret)).astype(np.int32)
    np.random.shuffle(arr)
    
    for i in range(len(ret)):
        pep =  ret[i]
        pep["imageID"] = int(arr[i])
    
    return ret

getYears(1970,5,5,"M")

[{'birth': {'day': 18, 'month': 1, 'year': 1960},
  'first': 'David',
  'gender': 'M',
  'imageID': 2,
  'last': 'Chapa',
  'middle': 'William',
  'name': 'David W. Chapa'},
 {'birth': {'day': 16, 'month': 12, 'year': 1962},
  'first': 'Kelly',
  'gender': 'M',
  'imageID': 4,
  'last': 'Kalish',
  'middle': 'Larry',
  'name': 'Kelly L. Kalish'},
 {'birth': {'day': 8, 'month': 4, 'year': 1964},
  'first': 'Wesley',
  'gender': 'M',
  'imageID': 1,
  'last': 'Wilson',
  'middle': 'Mike',
  'name': 'Wesley M. Wilson'},
 {'birth': {'day': 2, 'month': 8, 'year': 1973},
  'first': 'Charles',
  'gender': 'M',
  'imageID': 0,
  'last': 'Willis',
  'middle': 'Nicholas',
  'name': 'Charles N. Willis'},
 {'birth': {'day': 26, 'month': 11, 'year': 1973},
  'first': 'Brian',
  'gender': 'M',
  'imageID': 3,
  'last': 'Taylor',
  'middle': 'Derek',
  'name': 'Brian D. Taylor'}]

In [ ]:
## ouput

In [23]:
import json
import uuid

In [50]:
people = getYears(1965,7,50,"M")
people.extend( getYears(1970,5,50,"F"))
lookup = {}


for i in range(len(people)):
    key = str(uuid.uuid4())
    pep =  people[i]
    pep["id"] = key
    lookup[key] = pep



with open('data.txt', 'w') as outfile:
    json.dump({"people":people},outfile,indent=4, sort_keys=True)


In [39]:
arr = np.arange(10).astype(int)
np.random.shuffle(arr)
arr

array([8, 3, 1, 2, 6, 4, 5, 7, 0, 9])